In [1]:
%matplotlib widget
# %matplotlib inline
# %matplotlib qt 

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
from shapely.geometry import shape, GeometryCollection
from pyproj import CRS
from tqdm.notebook import tqdm
from tqdm import trange

In [6]:
import ee
ee.Initialize()

In [71]:
df = pd.read_parquet('../data/out/icesat-2-DK.parquet')

## Query water occurrence values for all ICESat-2 points

In [58]:
image = ee.Image('users/gena/eo-bathymetry/NDWI/NDWI-4521-v9')

In [53]:
print(image.reduceRegion(reducer=ee.Reducer.minMax(), geometry=image.geometry(), scale=90).getInfo())

{'water_score_max': 0.15000000000000024, 'water_score_min': -0.10000000000000006}


In [59]:
df.x

0          55.589424
1          55.588817
2          55.586167
3          55.586059
4          55.586015
             ...    
5583283    55.545941
5583284    55.546439
5583285    55.546509
5583286    55.546560
5583287    55.560946
Name: x, Length: 5583288, dtype: float64

In [62]:
df.series.unique()

array(['Noise', 'Buffer', 'Low', 'Medium', 'High'], dtype=object)

In [64]:
df_high = df[df.series=='High']

In [73]:
for i, row in df_high.iterrows():
    print(row)
    break

track_id            69
date        2019-01-01
beam              gt3r
series            High
x              55.5937
y               8.5166
h              80.1722
Name: 665, dtype: object


In [60]:
import geemap.eefolium as emap
# import geemap.geemap as emap
Map = emap.Map()
Map.addLayer(image, { 'min': -0.10, 'max': 0.15, 'palette': ['ffffff', '000000'] })
Map.setCenter(8.5199, 55.3184, 11)
Map.addLayerControl()
Map